## Цель выявить закономерность между действиями и скрытыми картами оппонентов (комбинации)
 - 
 - https://github.com/julianandrews/flopferret/blob/master/flopferret/board_texture.py

In [1]:
import re
import pandas as pd
from datetime import datetime
import numpy as np

import eval7, pprint
from phevaluator import evaluate_cards
from poker.room.pokerstars import PokerStarsHandHistory
from poker.hand import Hand, Combo
from poker.constants import Action, Position
from poker import handhistory

import matplotlib.pyplot as plt
plt.style.use('dark_background') #стиль графиков

import sentry_sdk

In [2]:
%%time

# Создали словарь
data_pd = {'TypeOfRoom': [], 'HandId': [], 'Tournament': [], 'Level': [], 'Date': [], 'Hour': [],'TimeZone': [], 'Table': [], 'bain': [], 'Players': [],  
           'HandHiro': [], 'Board':[], 'preflop_raises':[], 'preflop_calls':[], 'preflop_allin':[], 'flop_bets':[], 'flop_raises':[], 'flop_calls':[], 
           'flop_allin':[], 'turn_bets':[], 'turn_raises':[], 'turn_calls':[], 'turn_allin':[], 'river_bets':[], 'river_raises':[], 'river_calls':[], 'river_allin':[]} #

# Открываем файл
with open('Poker_/14437.txt', 'r',  encoding="utf8") as f: #  Poker_\613k.txt
    data_r = f.read()

iterr = re.findall("\n\n\n", data_r).count('\n\n\n') # задаём колличесвто итераций
print(iterr)

data = ""
iter_SHOW = 0

for i in range(iterr+1):
    if data_r.split("\n\n\n")[i].count('SHOW DOWN'):
        data = data + data_r.split("\n\n\n")[i] + "\n\n\n"
        iter_SHOW += 1
    else:
        continue
print(iter_SHOW)  

# данные
result = re.findall('(\w+) Hand #(\d*): Tournament #(\d*),(.*) Level .* \((.*)\) - (\d{4}/\d+/\d+) (\d+\D\d+\D\d+) (\w*)', data)
Tables = re.findall('Table \'.*\' (.*)-max Seat', data)
bains = re.findall('(\D\d+\.\d+\+\D\d+\.\d+)', data)
HandHiros = re.findall("Dealt to .* \D(.. ..)\D", data)
Boards = re.findall("Board \[(.*)\]", data)
#preflop_raises_ = re.findall("raises", data) # "raises"g

def preflop_acti(self):
    preflop_ = re.search("Dealt to .* \[(.*)\]\\n(.*)\*{3} FLOP", self, re.DOTALL)
    # handhiro = preflop_.group(1)
    action_preflop_raises= preflop_.group(2).count('raises')
    action_preflop_calls = preflop_.group(2).count('calls')
    action_preflop_allin = preflop_.group(2).count('all-in')
    return action_preflop_raises,action_preflop_calls, action_preflop_allin

def flop_acti(self):
    flop_ = re.search("FLOP \*{3} \[(.*)\]\\n(.*)\*{3} TU", self, re.DOTALL)
    #hand_flop = flop_.group(1)
    action_flop_bets = flop_.group(2).count('bets') 
    action_flop_raises= flop_.group(2).count('raises')
    action_flop_allin = flop_.group(2).count('all-in') 
    action_flop_calls = flop_.group(2).count('calls')
    return action_flop_bets, action_flop_raises, action_flop_calls, action_flop_allin

def turn_acti(self):
    turn_ = re.search("TURN \*{3} \[.*\] \[(.*)\](.*)\*{3} RI", self, re.DOTALL)
    #hand_turn = turn_.group(1)
    action_turn_bets = turn_.group(2).count('bets')
    action_turn_raises= turn_.group(2).count('raises')
    action_turn_allin = turn_.group(2).count('all-in')
    action_turn_calls = turn_.group(2).count('calls')
    return action_turn_bets, action_turn_raises, action_turn_calls, action_turn_allin

def river_acti(self):
    river_ = re.search("RIVER \*{3} \[.*\] \[(.*)\](.*)\\n\*{3} SHOW", self, re.DOTALL)
    #hand_river = river_.group(1)
    action_river_bets = river_.group(2).count('bets')
    action_river_raises= river_.group(2).count('raises')
    action_river_allin = river_.group(2).count('all-in')
    action_river_calls = river_.group(2).count('calls')
    return action_river_bets, action_river_raises, action_river_calls, action_river_allin

preflop_raisess = []
preflop_callss = [] 
preflop_allins = []
flop_betss = []  
flop_raisess = [] 
flop_callss = []
flop_allins = []
turn_betss = []  
turn_raisess = [] 
turn_callss = []
turn_allins = []
river_betss = []  
river_raisess = [] 
river_callss = []
river_allins = []
    
for y in range(iter_SHOW):
    data_n = data.split("\n\n\n")[y] # разбивает сплошной текст на раздачи
# ПРЕФЛОП    
    if data_n.count('raises'):
        preflop_raisess.append(preflop_acti(data_n)[0])
    else:
        preflop_raisess.append(0)
    if data_n.count('calls'):
        preflop_callss.append(preflop_acti(data_n)[1])
    else:
        preflop_callss.append(0)
    if data_n.count('all-in'):
        preflop_allins.append(preflop_acti(data_n)[2])
    else:
        preflop_allins.append(0)
     # ФЛОП        
    if data_n.count('bets'):
        flop_betss.append(flop_acti(data_n)[0])
    else:
        flop_betss.append(0)    
    if data_n.count('raises'):
        flop_raisess.append(flop_acti(data_n)[1])
    else:
        flop_raisess.append(0)
    if data_n.count('calls'):
        flop_callss.append(flop_acti(data_n)[2])
    else:
        flop_callss.append(0)
    if data_n.count('all-in'):
        flop_allins.append(flop_acti(data_n)[3])
    else:
        flop_allins.append(0)    
    # ТЁРН        
    if data_n.count('bets'):
        turn_betss.append(turn_acti(data_n)[0])
    else:
        turn_betss.append(0)    
    if data_n.count('raises'):
        turn_raisess.append(turn_acti(data_n)[1])
    else:
        turn_raisess.append(0)
    if data_n.count('calls'):
        turn_callss.append(turn_acti(data_n)[2])
    else:
        turn_callss.append(0)
    if data_n.count('all-in'):
        turn_allins.append(turn_acti(data_n)[3])
    else:
        turn_allins.append(0)    
    # РИВЕР        
    if data_n.count('bets'):
        river_betss.append(river_acti(data_n)[0])
    else:
        river_betss.append(0)    
    if data_n.count('raises'):
        river_raisess.append(river_acti(data_n)[1])
    else:
        river_raisess.append(0)
    if data_n.count('calls'):
        river_callss.append(river_acti(data_n)[2])
    else:
        river_callss.append(0)
    if data_n.count('all-in'):
        river_allins.append(river_acti(data_n)[3])
    else:
        river_allins.append(0)    
        
# Считаем сколко фишек
s = re.findall('Seat (\d).*chips', data)
s = [int(x) for x in s]
seats = []
length = 1
for i in range(len(s)-1):
    if s[i+1] < s[i]:
        seats.append(length)
        length = 1
    else:
        length += 1
seats.append(length)

# ЗАполнЯем словарь данными
for r, table, bain, seat, handHiro, Board ,preflop_raises, preflop_calls, preflop_allin, flop_bets, flop_raises, flop_calls, flop_allin, \
    turn_bets, turn_raises, turn_calls, turn_allin, river_bets, river_raises, river_calls, river_allin \
                in zip(result, Tables, bains, seats, HandHiros, Boards, preflop_raisess, preflop_callss, preflop_allins, flop_betss, flop_raisess, flop_callss, 
                       flop_allins, turn_betss, turn_raisess, turn_callss, turn_allins, river_betss, river_raisess, river_callss, river_allins): 
    
    data_pd['TypeOfRoom'].append(r[0])
    data_pd['HandId'].append(r[1])
    data_pd['Tournament'].append(r[2])
    # data_pd['info'].append(r[3])
    data_pd['Level'].append(r[4])
    data_pd['Date'].append(r[5])
    data_pd['Hour'].append(r[6])
    data_pd['TimeZone'].append(r[7])
    # data_pd['TimeZoneAbb'].append(r[8])
    data_pd['Table'].append(table) #table
    data_pd['Players'].append(seat)
    data_pd['bain'].append(bain)
    data_pd['HandHiro'].append(handHiro)
    data_pd['Board'].append(Board)
    data_pd['preflop_raises'].append(preflop_raises)
    #data_pd['showed_won'].append(showed_won)
    data_pd['preflop_calls'].append(preflop_calls)
    data_pd['preflop_allin'].append(preflop_allin)
    data_pd['flop_bets'].append(flop_bets)
    data_pd['flop_raises'].append(flop_raises)
    data_pd['flop_calls'].append(flop_calls)
    data_pd['flop_allin'].append(flop_allin)
    data_pd['turn_bets'].append(turn_bets)
    data_pd['turn_raises'].append(turn_raises)
    data_pd['turn_calls'].append(turn_calls)
    data_pd['turn_allin'].append(turn_allin)
    data_pd['river_bets'].append(river_bets)
    data_pd['river_raises'].append(river_raises)
    data_pd['river_calls'].append(river_calls)
    data_pd['river_allin'].append(river_allin)
                                    

# Создаём ДатаФрейм
df = pd.DataFrame(data_pd)
df.head()

14437
5606
CPU times: total: 7min 58s
Wall time: 8min 47s


,TypeOfRoom,HandId,Tournament,Level,Date,Hour,TimeZone,Table,bain,Players,...,flop_calls,flop_allin,turn_bets,turn_raises,turn_calls,turn_allin,river_bets,river_raises,river_calls,river_allin
0,PokerStars,219890250295,3038500993,150/300,2020/10/31,0:00:00,MSK,9,$13.70+$1.30,4,...,0,0,0,0,0,0,0,0,0,0
1,PokerStars,219890415599,3038500993,200/400,2020/10/31,0:04:11,MSK,9,$13.70+$1.30,3,...,0,0,0,0,0,0,0,0,0,0
2,PokerStars,219890439999,3038500993,200/400,2020/10/31,0:04:47,MSK,9,$13.70+$1.30,2,...,1,0,1,0,1,0,0,0,0,0
3,PokerStars,219890554035,3038500993,200/400,2020/10/31,0:07:35,MSK,9,$13.70+$1.30,2,...,0,0,0,0,0,0,0,0,0,0
4,PokerStars,219890623054,3038500993,200/400,2020/10/31,0:09:16,MSK,9,$13.70+$1.30,2,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
Board_flops = []

for i in range(iterr):
    data_n = data.split("\n\n\n")[i] # разбивает сплошной текст на раздачи
    
    if data_n.count('FLOP'): # карты флопа
        Board_flops.append(flop_acti(data_n)[0])
    else:
        continue
        #Board_flops.append(0)
        
data_n   

In [ ]:
flop_acti(data_n)[0]

In [ ]:
Board_flops.append(flop_acti(data_n)[0])
Board_flops

In [ ]:
# Открываем файл
with open('dim10.txt', 'r') as f: #  Poker_\SNG dimetruk 1-54133.txt
    data = f.read()
data

In [ ]:
iterr = re.findall("\n\n\n", data).count('\n\n\n') # задаём колличесвто итераций

preflop_raises_= []

for i in range(iterr):
    data_n = data.split("\n\n\n")[i]
    if data_n.count('SHOW DOWN'):
        print(preflop_(data_n)[1])
        preflop_raises_.append(preflop_(data_n)[1])    
    
    
preflop_raises_    

In [ ]:
data_n = data.split("\n\n\n")[0]
data_n.count('SHOW DOWN')

In [ ]:
def preflop_acti(self):
    preflop_ = re.search("Dealt to .* \[(.*)\]\\n(.*)\*{3} FLOP", self, re.DOTALL)
    handhiro = preflop_.group(1)
    action_preflop_raises= preflop_.group(2).count('raises')
    action_preflop_calls = preflop_.group(2).count('calls')
    action_preflop_allin = preflop_.group(2).count('all-in')
    return handhiro, action_preflop_raises,action_preflop_calls, action_preflop_allin

preflop_acti(data_n)[0]

In [ ]:
flop_ = re.search("FLOP \*{3} \[(.*)]\\n(.*)\*{3} TU", data_n, re.DOTALL)
flop_

In [ ]:
def flop_acti(self):
    flop_ = re.search("FLOP \*{3} \[(.*)]\\n(.*)\*{3}", self, re.DOTALL)
    hand_flop = flop_.group(1)
    action_flop_bets = flop_.group(2).count('bets')
    action_flop_raises= flop_.group(2).count('raises')
    action_flop_allin = flop_.group(2).count('all-in') #count('all-in')
    action_flop_calls = flop_.group(2).count('calls')
    return hand_flop, action_flop_bets, action_flop_raises, action_flop_allin, action_flop_calls

flop_acti(data_n)[0]

In [ ]:
def turn_acti(self):
    turn_ = re.search("TURN \*{3} \[.*\] \[(.*)\](.*)\*{3} RI", self, re.DOTALL)
    hand_turn = turn_.group(1)
    action_turn_bets = turn_.group(2).count('bets')
    action_turn_raises= turn_.group(2).count('raises')
    action_turn_allin = turn_.group(2).count('all-in')
    action_turn_calls = turn_.group(2).count('calls')
    return action_turn_bets, action_turn_raises, action_turn_allin, action_turn_calls

In [ ]:
def river_acti(self):
    river_ = re.search("RIVER \*{3} \[.*\] \[(.*)\](.*)\\n\*{3} SHOW", self, re.DOTALL)
    hand_river = river_.group(1)
    action_river_bets = river_.group(2).count('bets')
    action_river_raises= river_.group(2).count('raises')
    action_river_allin = river_.group(2).count('all-in')
    action_river_calls = river_.group(2).count('calls')
    return action_river_bets, action_river_raises, action_river_allin, action_river_calls

In [ ]:
SHOW_DOWN = re.search("SHOW DOWN.*Seat \d+: .+? showed \[(.*)\] and won", data_n, re.DOTALL)
SHOW_DOWN.group(1)

In [ ]:
from decimal import Decimal, DecimalException
import sys, os

with open('dim10.txt', 'r') as f: #  Poker_\SNG dimetruk 1-54133.txt  Poker_\SNG dimetruk 1-14437.txt
    data4 = f.read()

seats = []
pref_calls = 0
pref_raisess= []
pref_allins = []
flop_calls = []
flop_checks = []

length_calls = 0
length_raisess = 0
length_allins = 0
length_f_calls = 0
length_f_checks = 0

for i in range(len(data4.split("\n\n\n"))): # -1 для предупреждения ошибки, выход из цикда без последнего элемента
    try:
        #raise NotImplementedError("No error")
        distribut = data4.split("\n\n\n")[i]
        hhh = PokerStarsHandHistory(distribut)
        hhh.parse()
        
        pfa = str(hhh.preflop_actions)
        fa = str(hhh.flop.actions)

        seat = len(hhh.preflop_actions)
        seats.append(seat)
        
        pr_call = re.findall("calls", pfa)
        pr_rais = re.findall("raises", pfa)
        pr_allin = re.findall("all-in", pfa)

        f_check = re.findall("check", fa)
        f_call = re.findall("call", fa)
        f_rais = re.findall("raise", fa)
        f_allin = re.findall("bet", fa)
        #print(pr_allin)

            
        if re.findall("calls", pfa):
            pref_calls += 1
            continue 
        
        
        
    except AttributeError:
        data4.split("\n\n\n")
        
    # except RuntimeError as ru:
    #     print(ru)
        
#     except Exception as e:
#         print(e)    


# try:
#     raise NotImplementedError("No error")
# except Exception as e:
#     exc_type, exc_obj, exc_tb = sys.exc_info()
#     fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
#     print(exc_type, fname, exc_tb.tb_lineno)

pref_calls

In [ ]:
flop_checks

In [ ]:
with open('tochka.txt') as f:
    file_data = [[i for i in line.split('.')] for line in f]
    
file_data    

In [ ]:
s = re.findall('Seat (\d).*chips', data4)
s = [int(x) for x in s]
seats = []
length = 1
for i in range(len(s)-1):
    if s[i+1] < s[i]:
        seats.append(length)
        length = 1
    else:
        length += 1
seats.append(length)
seats

In [ ]:
my_string = data
i=my_string.split("\n\n\n")[11]

hhh = PokerStarsHandHistory(i)
hhh.parse()
pfa = str(hhh.preflop_actions)
fa = str(hhh.flop.actions)

pr_rais = re.findall("raises", pfa)
pr_call = re.findall("calls", pfa)

f_call = re.findall("checks", fa)

def preflop_action(p):
    p1 = len(p)
    action = []
    length = 0
    for y in range(p1):
        if y==0:
            length = 1
        else:
            length += 1  
    action.append(length)
    return action


print(preflop_action(f_call))

In [ ]:
s = re.findall('Seat (\d).*chips', data)
s1 = [int(x) for x in s]
seats = []
length = 1
for i in range(len(s1)-1):
    if s1[i+1] < s1[i]:
        seats.append(length)
        length = 1
    else:
        length += 1        
seats.append(length)     
seats


In [ ]:
s = re.findall('Seat (\d).*chips', data)
s = [int(x) for x in s]
seats = []
length = 1
for i in range(len(s)-1):
    if s[i+1] < s[i]:
        seats.append(length)
        length = 1
    else:
        length += 1

In [ ]:
import pandas as pd
from datagen import DataGenerator, EmptyLine
from poker.room.pokerstars import PokerStarsHandHistory
from functions import FindFold, GetFiles, GetShowers
#IMPORT DATAGEN AQUI

database = pd.DataFrame()

files = GetFiles('hand_texts')

database = []
for hand_text in files:
    # as a sidenote, with contexts are helpful for these:
    with open('dim100.txt', 'r') as text:
        b=text.read()

    hands=b.split("\n\n\n\n\n")

    for i in range(1,len(hands)):
        try:
            hh = PokerStarsHandHistory(unicode(hands[i]))
            hh.parse()
            fold = FindFold(hh)

            if fold == 'showdown':
                for shower in GetShowers(hh): 
                    database.append(DataGenerator(hh,shower,hand_text,i))
                    print('Success in parsing iteration ' + str(i) + ' from file' + hand_text)

        except:
            print('PARSER ERROR ON ITERATION [[' + str(i) + ']] FROM FILE [[' + hand_text + ']]')
            database.append(EmptyLine(hand_text,i))

pd.DataFrame(database).to_csv('database2.csv')

In [ ]:
np.random.seed(2)
data = []
for i in range(8):
    data.append(dict(zip(np.random.choice(10, replace=False, size=5),
                         np.random.randint(15, size=5))))
df = pd.DataFrame(data)
print(df)

In [ ]:
len(re.findall(pattern, string_to_search))

In [ ]:
"[^\D]*(raises)(?!(\D FLOP \D))[^\D]*"g

## Преобразуем строковые данные

In [ ]:
date = data_pd['Date']
time = data_pd['Hour']

dataf["Date"] = pd.to_datetime(data_pd['Date'])
dataf["Time"] = pd.to_timedelta(data_pd['Hour'])

dataf["DateTime"] = dataf["Date"] + dataf["Time"]
dataf["DateTime"]

# filter by single day
dataf_m = dataf[dataf['DateTime'].dt.strftime('%d') == '06']

dataf.Tournament = dataf.Tournament.astype('int64')
dataf = dataf.astype({'HandId': np.int64, 'Tournament': np.int64})

In [ ]:
dataf_m

In [ ]:
nov_mar_series = pd.Series(pd.date_range("2013-11-15", "2014-03-15"))
#create timestamp without year
nov_mar_no_year = nov_mar_series.map(lambda x: x.strftime("%m-%d"))
nov_mar_no_year

In [ ]:
# #add a yearless timestamp to the dataframe
# df["no_year"] = df['nov_mar_series'].map(lambda x: x.strftime("%m-%d"))
# no_year_mask = df['no_year'].isin(nov_mar_no_year)
# df[no_year_mask]

## Фильтруем турниры по числу рук выше среднего (> mean)

In [ ]:
data_filter = dataf[['Tournament', 'HandId','DateTime', 'Time', 'HandHiro', 'Board']]
data_filter.head()

In [ ]:
# data_filter.groupby(['Tournament']).agg({'HandId': "count"}).reset_index()

In [ ]:
#  Фильтруйте value_counts с помощью isin  https://softhints.com/pandas-how-to-filter-results-of-value_counts/
df = data_filter[data_filter['Tournament'].isin(data_filter['Tournament'].
                                                 value_counts()[data_filter['Tournament'].value_counts()>100].index)]
df

In [ ]:
#  Используйте group by и lambda для имитации фильтра для value_counts()
#df = data_filter.groupby('Tournament').filter(lambda x: len(x) > (data_filter['Tournament'].value_counts().mean()))


In [ ]:
df.Tournament.value_counts()

In [ ]:
# Prepare Data
df_plot = df.groupby('Tournament').size()

# Make the plot with pandas
df_plot.plot(kind='pie', subplots=True, figsize=(8, 8))
plt.title("Колличество турниров")
plt.ylabel("")
plt.show()

In [ ]:
# объеденяем руку и борд, выводим название комбинации

# df['hand'] = df['HandHiro'] + " " + df['Board']
# # hand = df['hand'].values[7].split()
# # print(hand)

# hand_ev = [eval7.Card(s) for s in (df['hand'].values[7].split())]
# rang = eval7.evaluate(hand_ev)   # ранг комбинации
# # print(rang)

# eval7s = eval7.handtype(rang)  #  название комбинации

# df['rang'] = eval7.evaluate(hand_ev)
# df['eval7s'] =  eval7.handtype(rang)

In [ ]:
# def new_type(row):
#     if row.traffic_source.lower() in ('yandex', 'google'):
#         return 'organic'
#     elif row.traffic_source.lower() in ('paid', 'email'):
#         if row.region.lower() == 'russia':
#             return 'ad'
#         else:
#             return 'other'

In [ ]:
# log['source_type'] = log.apply(new_type, axis = 1)
# log.head(12)

In [ ]:
# log.loc[log.traffic_source.isin(['yandex', 'google']), 'traffic_type'] = 'organic' 

In [ ]:
df['hand_rang'] = df['HandHiro'] + " " + df['Board']
df['hand_rang']

print(type(df['hand_rang']), type(df['hand_rang'].values[7].split()))
df['hand_rang'].values[0].split()

In [ ]:
# df['hand_rang'].to_list()
# df['hand_rang'].tolist()
# np.array(df['hand_rang'].tolist())
# Ls = list(df['hand_rang'])
ls = df['hand_rang'].str.split()
ls

In [ ]:
# напишем функцию получения ранг комбинации

def evaluate_(hand):
    """
    На вход получает карты, на выход 
    число ранг по eval7
    """
    hand_ev = [eval7.Card(s) for s in (hand)]
    return eval7.evaluate(hand_ev),hand_ev

a = ('Jh', 'Td', '8h', '8c', '7c', '3h', '6d')
evaluate_(a)

In [ ]:
# напишем функцию получения названия ранга

def handtype_(rang):
    """
    На вход получает ранг, на выход 
    название комбинации
    """
    return eval7.handtype(rang)

handtype_(17209424)

In [ ]:
# Применим функцию к столбцу hand  и запишем в новый столбец rang_namber

df['rang_namber'] = df['hand_rang'].str.split().apply(evaluate_)
df['handtypes'] = df['rang_namber'].apply(handtype_)
df[['rang_namber', 'handtypes','HandHiro', 'Board']].head(20)
# df['filter'] = df.loc[df.handtypes.str.contains('Two Pair', 'Trips', 'Straight', 'Full House', 'Quads', 'Straight Flush'), 'rang_namber']

In [ ]:
# df.sort_values(by=['handtypes', 'rang_namber'])

In [ ]:
# df.groupby('handtypes')['rang_namber'].value_counts()

### Создадим сводную таблицу, что-бы проверить наличие аномалий

In [ ]:
# filter by single day
dataf_m = dataf[dataf['DateTime'].dt.strftime('%d') == '06']

In [ ]:
df[['Tournament', 'handtypes', 'rang_namber']].head()  # применим фильтер по столбцам

In [ ]:
pd.pivot_table(df, index=["Tournament", "handtypes"])

In [ ]:
# явно определив столбцы, которые нам нужны,
pd.pivot_table(df,
               index=["Tournament", "handtypes"],
               values=["rang_namber"],
               aggfunc= len)   # aggfunc=[np.mean, len])

In [ ]:
df.Tournament.value_counts()

In [ ]:
# df['Time'].value_counts()

In [ ]:
# Prepare Data
df_plot = df.groupby('handtypes').size()

# Make the plot with pandas
df_plot.plot(kind='area', subplots=True, figsize=(8, 8))
plt.title("Колличество комбинаций")
plt.ylabel("")
plt.show()

In [ ]:
# df['DateTime'] = df['DateTime'][(df['DateTime'] > '2018-10-01 20:00:00') & (df['DateTime'] < '2022-10-02 00:00:00')]
# df['DateTime']


In [ ]:
# отфильтруем комбинации "старшая карта" и "пара"
df['no_par'] = df['rang_namber'][df['rang_namber'] > 50000000] #[((df['rang_namber'] > 50000000) & (df['rang_namber'] < 60000000))]
df = df.dropna()
df[['Time', 'no_par', 'handtypes']].head(10)
# df['filter_Two Pair'] = df['handtypes'][df['handtypes'] == 'Two Pair']
# df['filter_Two Pair']

In [ ]:
df.isnull().sum()

In [ ]:
# преобразуем время во временной ряд

In [ ]:


idx = df['Time']
 # datetime.timedelta(seconds=24*60*60).total_seconds()

# second = df['DateTime'].dt.total_seconds()
# second

# idx = pd.date_range(df['DateTime'], periods=5, freq="H")
# idx
tim = pd.Series(range(len(idx)), index=idx)

# ts.resample("2H").mean()
tim

In [ ]:
# df = df.set_index("Time")
# df

In [ ]:

import matplotlib.pyplot as plt
 
# Using a inbuilt style to change
# the look and feel of the plot
plt.style.use("classic")
 
# setting figure size to 12, 10
plt.figure(figsize=(12, 10))
 
# Labelling the axes and setting
# a title
plt.xlabel("Time")
plt.ylabel("Values")
plt.title("Sample Time Series Plot")
 
# plotting the "A" column alone
plt.plot(df["handtypes"])

In [ ]:
df.plot(subplots=True, figsize=(12, 15))

In [ ]:
# # # idx = pd.date_range('2019-01-01', periods=48, freq='H')
# # x = pd.Series(range(len(idx)), index=idx)
# y = tim
# # add 'hour' and 'day' columns in the dataframe
# d = {'x': x, 'y': y, 'hour': idx.hour, 'day': idx.day}
# df = pd.DataFrame(data=d)
# # use 'hour' as x axis to plot, and 'day' as marker color
# df.plot.scatter(x='hour', y='y', c='day', colormap='rainbow')
# plt.show()

In [ ]:
data.dtypes

In [ ]:
# установитьdatetimeОбъект в качестве индекса кадра данных
data = data.set_index('DateTime')
data.head()

In [ ]:

# data.plot(grid=True)

data['filter'].plot(kind='bar')

# import matplotlib.pyplot as plt
# from matplotlib.dates import DateFormatter
# # Группировка за месяц
# df_month = data.resample("M").mean()

# fig, ax = plt.subplots(figsize=(10, 6))
# ax.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
# ax.bar(
#     df_month['2018':].index, 
#     df_month.loc['2016':, "filter"], 
#     width=25, align='center')

In [ ]:

data.plot(kind='scatter', 
        x='DateTime', 
        y='filter', 
        title='Связь между жёсткостью воды и средней годовой смертностью')